In [417]:
pip install langchain


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [418]:
pip install langchain-google-genai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [419]:
from dotenv import load_dotenv
load_dotenv()

from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langgraph.types import interrupt, Command
from langgraph.checkpoint.memory import MemorySaver

In [420]:
class workflowState(TypedDict):
    preferences: Annotated[list[str], add_messages]
    breakfast: Annotated[list[str], add_messages]
    lunch: Annotated[list[str], add_messages]
    dinner: Annotated[list[str], add_messages]


In [421]:
llm = init_chat_model("google_genai:gemini-2.0-flash")

In [422]:
def get_preferences(state: workflowState):
    state["preferences"] = "vegetarian"
    return { "preferences":  state["preferences"] }

In [423]:
def gen_breakfast(state: workflowState):
    result = llm.invoke(f"generate {state["preferences"]} breakfast meal coma seperated small list")
    state["breakfast"] = result.content
    return { "breakfast":  state["breakfast"] }

In [424]:
def gen_lunch(state: workflowState):
    result = llm.invoke(f"generate {state["preferences"]} lunch meal coma seperated small list")
    state["lunch"] = result.content
    return { "lunch":  state["lunch"] }

In [425]:
def gen_dinner(state: workflowState):
    result = llm.invoke(f"generate {state["preferences"]} dinner meal coma seperated small list")
    state["dinner"] = result.content
    return { "dinner":  state["dinner"] }

In [426]:
def review(state: workflowState):
    print("\n Generated meal plan")
    print("Breakfast:", state["breakfast"])
    print("Lunch:", state["lunch"])
    print("Dinner:", state["dinner"])

    data = f"Generated mean plan\n Breakfast: {state["breakfast"]}, Lunch: {state["lunch"]}, Dinner: {state["dinner"]} review and approve the meal"
    decision = interrupt(data)
    if decision == "yes":
        return Command(
            goto="finalize"
        )
    else:
        nodevalue = input('Enter the step you want to go')
        return Command(
            goto=nodevalue
        )

    

In [427]:
def finalize(state: workflowState):
    print(f"meal finalized Breakfast: {state["breakfast"]}, Lunch: {state["lunch"]}, Dinner: {state["dinner"]}")
    return state

In [428]:
def reject(state: workflowState):
    print(f"meal rejected Breakfast: {state["breakfast"]}, Lunch: {state["lunch"]}, Dinner: {state["dinner"]}")
    return state

In [429]:
graph = StateGraph(workflowState)

memory = MemorySaver()
graph.add_node("get_preferences", get_preferences)
graph.add_node("gen_breakfast", gen_breakfast)
graph.add_node("gen_lunch", gen_lunch)
graph.add_node("gen_dinner", gen_dinner)
graph.add_node("review", review)
graph.add_node("finalize", finalize)
# graph.add_node("reject", reject)

graph.add_edge(START, "get_preferences")
graph.add_edge("get_preferences", "gen_breakfast")
graph.add_edge("get_preferences", "gen_lunch")
graph.add_edge("get_preferences", "gen_dinner")

graph.add_edge("gen_breakfast", "review")
graph.add_edge("gen_lunch", "review")
graph.add_edge("gen_dinner", "review")

graph.add_edge("review", "finalize")
graph.add_edge("finalize", END)

workflow = graph.compile(checkpointer=memory)

In [430]:

config = {"configurable": {"thread_id": "1"}}
result = workflow.invoke({}, config=config, stream_mode="updates")
result


 Generated meal plan
Breakfast: [HumanMessage(content='Oatmeal with berries, Tofu scramble with spinach, Yogurt with granola and fruit, Avocado toast with everything bagel seasoning, Breakfast burrito with black beans and salsa', additional_kwargs={}, response_metadata={}, id='c32d0270-5ac6-4dec-b4ef-34c2982154d5')]
Lunch: [HumanMessage(content='Vegetable curry with rice, Lentil soup and whole-wheat bread, Quinoa salad with roasted vegetables, Black bean burgers on whole-grain buns, Pasta primavera', additional_kwargs={}, response_metadata={}, id='8a304864-7ccc-4034-9b9a-bb8293a766e8')]
Dinner: [HumanMessage(content='Vegetable curry, Lentil soup, Pasta primavera, Black bean burgers, Stuffed bell peppers, Vegetarian chili, Quinoa salad, Mushroom risotto, Tofu stir-fry', additional_kwargs={}, response_metadata={}, id='0093faa5-bd81-4a6f-a4db-4a6f958af73c')]


[{'get_preferences': {'preferences': 'vegetarian'}},
 {'gen_breakfast': {'breakfast': 'Oatmeal with berries, Tofu scramble with spinach, Yogurt with granola and fruit, Avocado toast with everything bagel seasoning, Breakfast burrito with black beans and salsa'}},
 {'gen_lunch': {'lunch': 'Vegetable curry with rice, Lentil soup and whole-wheat bread, Quinoa salad with roasted vegetables, Black bean burgers on whole-grain buns, Pasta primavera'}},
 {'gen_dinner': {'dinner': 'Vegetable curry, Lentil soup, Pasta primavera, Black bean burgers, Stuffed bell peppers, Vegetarian chili, Quinoa salad, Mushroom risotto, Tofu stir-fry'}},
 {'__interrupt__': (Interrupt(value="Generated mean plan\n Breakfast: [HumanMessage(content='Oatmeal with berries, Tofu scramble with spinach, Yogurt with granola and fruit, Avocado toast with everything bagel seasoning, Breakfast burrito with black beans and salsa', additional_kwargs={}, response_metadata={}, id='c32d0270-5ac6-4dec-b4ef-34c2982154d5')], Lunch: [

In [431]:
snapshot = workflow.get_state(config)
snapshot.next

('review',)

In [432]:
for chunk in workflow.stream({}, config=config):
    for node_id, value in chunk.items():

        if(node_id == "__interrupt__"):
            while True: 
                decision = input("Approve meal plan? (yes/no): ").strip().lower()

                result = workflow.invoke(Command(resume=decision), config=config, stream_mode="updates")
                result
            
                if decision.lower() == "yes":
                    break


 Generated meal plan
Breakfast: [HumanMessage(content='Oatmeal with berries, Tofu scramble with spinach, Yogurt with granola and fruit, Avocado toast with everything bagel seasoning, Breakfast burrito with black beans and salsa', additional_kwargs={}, response_metadata={}, id='c32d0270-5ac6-4dec-b4ef-34c2982154d5'), HumanMessage(content='Omelet, Pancakes, Waffles, French Toast, Oatmeal', additional_kwargs={}, response_metadata={}, id='cf5b986c-6888-445e-84ea-9417310b3354')]
Lunch: [HumanMessage(content='Vegetable curry with rice, Lentil soup and whole-wheat bread, Quinoa salad with roasted vegetables, Black bean burgers on whole-grain buns, Pasta primavera', additional_kwargs={}, response_metadata={}, id='8a304864-7ccc-4034-9b9a-bb8293a766e8'), HumanMessage(content='vegetarian, pasta salad, vegetarian chili, soup, sandwich, salad', additional_kwargs={}, response_metadata={}, id='86435cb1-f0ff-4480-bd3d-3e2bb5281d6c')]
Dinner: [HumanMessage(content='Vegetable curry, Lentil soup, Pasta 

In [433]:
decision = input("Approve meal plan? (yes/no): ").strip().lower()
result = workflow.invoke(Command(resume=decision), config=config, stream_mode="updates")
result

[]

In [434]:
step = input("Enter the step you want to go: ").strip().lower()
result = workflow.invoke(Command(goto=step), config=config, stream_mode="updates")
result

Task __input__ with path () wrote to unknown channel branch:to:yes, ignoring it.


[]

In [435]:
snapshot = workflow.get_state(config)
snapshot.next

Task __input__ with path () wrote to unknown channel branch:to:yes, ignoring it.


()

In [436]:
decision = input("Approve meal plan? (yes/no): ").strip().lower()
result = workflow.invoke(Command(resume=decision), config=config, stream_mode="updates")
result

Task __input__ with path () wrote to unknown channel branch:to:yes, ignoring it.


[]

In [437]:
result = workflow.invoke(Command(resume='finalize'), config=config, stream_mode="updates")
result

Task __input__ with path () wrote to unknown channel branch:to:yes, ignoring it.


[]

In [438]:


events = workflow.stream(None, config=config, stream_mode="values")
for event in events:
    print(event)

Task __input__ with path () wrote to unknown channel branch:to:yes, ignoring it.


{'preferences': [HumanMessage(content='vegetarian', additional_kwargs={}, response_metadata={}, id='81bf0f5c-aa41-4840-b9a7-5e25ff99fc6b'), HumanMessage(content='vegetarian', additional_kwargs={}, response_metadata={}, id='d814170c-c800-4efd-bdd5-b640f02b7438')], 'breakfast': [HumanMessage(content='Oatmeal with berries, Tofu scramble with spinach, Yogurt with granola and fruit, Avocado toast with everything bagel seasoning, Breakfast burrito with black beans and salsa', additional_kwargs={}, response_metadata={}, id='c32d0270-5ac6-4dec-b4ef-34c2982154d5'), HumanMessage(content='Omelet, Pancakes, Waffles, French Toast, Oatmeal', additional_kwargs={}, response_metadata={}, id='cf5b986c-6888-445e-84ea-9417310b3354')], 'lunch': [HumanMessage(content='Vegetable curry with rice, Lentil soup and whole-wheat bread, Quinoa salad with roasted vegetables, Black bean burgers on whole-grain buns, Pasta primavera', additional_kwargs={}, response_metadata={}, id='8a304864-7ccc-4034-9b9a-bb8293a766e8'